** Explore Bus Trips! **

This app allow you to do Data exploration.

All you need to do is specifiy the data path (or IP if streaming), and start to explore by using our interative dashboard, automatic visualizaion platform or quick sampled data viz!

*More detailed explanation below.

In [2]:

from IPython.display import Image
Image(url="https://drive.google.com/uc?id=1S13K-zCiK2nLgY9pdSSjgntxyOl-nm-x",  width=200)

**Dashboard**

To filter the Interactive Dashboard use the different filters below, or directly select \ click on the part you are interested in. 

Alternatively, use the query bar below (for example: "vehicleId :33317 or vehicleId :33318" to see only these vehicles).

In [4]:
!pip install libify
import libify
from pyspark.sql.functions import date_format
import pyspark.sql.functions as F

lab4_functions = libify.importer(globals(), '/Users/shubi@campus.technion.ac.il/Lab4_functions')

dbutils.widgets.dropdown("Stream?", "False", ['True', 'False'])
dbutils.widgets.text("IP / Path", '')

isStream = eval(dbutils.widgets.get("Stream?"))
path_ip = dbutils.widgets.get("IP / Path")

# Our check
# 10.0.0.30:9091
# /mnt/dacoursedatabricksstg/dacoursedatabricksdata/busFile 
df = lab4_functions.create_warehouse_df(streaming=isStream, USE_TAKE=True, min_date=None, max_date=None, path_ip=path_ip)
if df == 'problem':
    print('\n\n\n\n')
    print('Unexpected error in reading file.\n'
          'Please make sure the file exists, ' 
          'is of the correct format(json) and with the right schema.')
df = lab4_functions.hydrate_data(df)
df = lab4_functions.aggregate_data(df, streaming=isStream)
df = df.drop('congestionPercentage')
if isStream:
    df = df.withColumn('day', date_format("TimestampDay", 'yyyy-MM-dd')).drop("TimestampDay")
df = df.withColumn('isEventDate', F.col('isEventDate').cast('int'))\
       .withColumn('isPassedLocation', F.col('isPassedLocation').cast('int'))    
lab4_functions.write_to_elastic(df, 'group-data-to-explore', lab4_functions.elastic_settings_mappings_date, overwrite=True, streaming=isStream)

In [5]:
displayHTML("""<iframe src="https://da2020w-0001.eastus.cloudapp.azure.com:5601/goto/9ce95d281a2ae62b5b92202efa43a7ca" height="1013" width="1800"></iframe>""")

**Automatic visualizaion platform** - *visualizations with a single mouse gesture* - Drag & Drop a field of interest into the center part of the display. 

Then, pick one of smart suggestions of ways to visualize your data.

Alternatively, easily change the underlying query, visualization type or filters, all with an easy-to-use, intuitive UI!

In [7]:
displayHTML("""<iframe src="https://da2020w-0001.eastus.cloudapp.azure.com:5601/app/lens#/?_g=(filters:!(),refreshInterval:(pause:!f,value:30000),time:(from:'2017-03-01T10:56:37.000Z',to:'2018-09-11T18:10:43.000Z'))" height="1013" width="1800"></iframe>""")

**Visualization using sampled data**

This interface allows to quickly get insights about the general distribution of the data, and some statistics about it.
Even on large database, by selecting a sample size you can get fast results. 
Additionally, you can query the database and get approximate results as well. 

Updated in realtime.

In [9]:
displayHTML("""<iframe src="https://da2020w-0001.eastus.cloudapp.azure.com:5601/app/ml#/jobs/new_job/datavisualizer?_g=%28refreshInterval%3A%28pause%3A%21f%2Cvalue%3A30000%29%2Ctime%3A%28from%3A%272017-07-03T00%3A00%3A00.000Z%27%2Cto%3A%272018-02-13T00%3A00%3A00.000Z%27%29%29&index=b17555c0-574c-11eb-b032-278f6d866662" height="3000" width="1800"></iframe>""")

* Choose the data to run on by adding ip number if it is a streaming data, and a file path if it is a batch data.
* Run by pressing on the "update" button that appears at the top right of the app. (It might pop-up a massage, please press 'yes')
* -> The visualizations automaticaly will be updated! Now you can explore in a convient and fast way.
* Notes: 
  1. If the map is visualizations you might entered a wrong ip number.
  2. The running might take a while.
  3. If you see a pop-up massage "Run beyond time out" press it to continue.
  4. if you are getting error "Failed to construct kafka consumer" - you might entered a wrong ip number.